# Rap Music Analysis

In [1]:
import urllib.request
import urllib.parse
import urllib.error
from urllib.request import Request, urlopen
import requests

from bs4 import BeautifulSoup

import pandas as pd
import json

In [2]:
column_names = ['Title','Artist','Lyrics','Comments']

df = pd.DataFrame(columns = column_names)

In [4]:
def request_artist_info(artist_name, page):
    
    base_url = 'https://api.genius.com'
    GENIUS_API_TOKEN = 'VetJWKIjYbo8ujjg8V_PTQnqAkY8xN1fbajgaaTiON0rkgWFuI6X47dXPKCaZQLM'
    
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=50&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    
    return response      

In [5]:
response = request_artist_info('Drake', 1)
json_object = response.json()
json_formatted_str = json.dumps(json_object, indent=2)
print(json_formatted_str)

{
  "meta": {
    "status": 200
  },
  "response": {
    "hits": [
      {
        "highlights": [],
        "index": "song",
        "type": "song",
        "result": {
          "annotation_count": 20,
          "api_path": "/songs/3315890",
          "full_title": "God's Plan by\u00a0Drake",
          "header_image_thumbnail_url": "https://images.genius.com/921b81a80b404f357609b3884052312e.300x169x1.jpg",
          "header_image_url": "https://images.genius.com/921b81a80b404f357609b3884052312e.1000x563x1.jpg",
          "id": 3315890,
          "lyrics_owner_id": 1507505,
          "lyrics_state": "complete",
          "path": "/Drake-gods-plan-lyrics",
          "pyongs_count": 852,
          "song_art_image_thumbnail_url": "https://images.genius.com/efde8de93f83871f717e1019d453245c.300x300x1.png",
          "song_art_image_url": "https://images.genius.com/efde8de93f83871f717e1019d453245c.1000x1000x1.png",
          "stats": {
            "unreviewed_annotations": 0,
            "c

In [16]:
for hit in json['response']['hits']:
     hit['result']['primary_artist']['name'].lower()

TypeError: 'module' object is not subscriptable

In [6]:
def request_song_url(artist_name, song_cap):
    
    """ This function populates a list with urls to the songs by a chosen artist. 

    Args: 
        artist_name(str): name of the artist
        song_cap(int): number of urls/songs you want 
        
    Returns: 
        list of strings, each string is a url to a song by the artist
    
    """
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()       
        song_info = []
        
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
        
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs

In [7]:
request_song_url('drake',10)

Found 10 songs by drake


['https://genius.com/Drake-gods-plan-lyrics',
 'https://genius.com/Drake-in-my-feelings-lyrics',
 'https://genius.com/Drake-hotline-bling-lyrics',
 'https://genius.com/Drake-one-dance-lyrics',
 'https://genius.com/Drake-hold-on-were-going-home-lyrics',
 'https://genius.com/Drake-know-yourself-lyrics',
 'https://genius.com/Drake-back-to-back-lyrics',
 'https://genius.com/Drake-all-me-lyrics',
 'https://genius.com/Drake-fake-love-lyrics',
 'https://genius.com/Drake-from-time-lyrics']

In [8]:
first_hit = json_object['response']['hits'][0]

In [9]:
urls = request_song_url('Lana Del Rey',10)

urls

Found 10 songs by Lana Del Rey


['https://genius.com/Lana-del-rey-young-and-beautiful-lyrics',
 'https://genius.com/Lana-del-rey-love-lyrics',
 'https://genius.com/Lana-del-rey-summertime-sadness-lyrics',
 'https://genius.com/Lana-del-rey-lust-for-life-lyrics',
 'https://genius.com/The-weeknd-and-lana-del-rey-prisoner-lyrics',
 'https://genius.com/Lana-del-rey-born-to-die-lyrics',
 'https://genius.com/Lana-del-rey-high-by-the-beach-lyrics',
 'https://genius.com/Lana-del-rey-video-games-lyrics',
 'https://genius.com/Lana-del-rey-norman-fucking-rockwell-lyrics',
 'https://genius.com/Lana-del-rey-cherry-lyrics']

In [10]:
for url in urls: 
    
    req = Request(url, headers = {"User-Agent" : "Mozilla/5.0"})
    webpage = urlopen(req).read()

    # Create a BeautifulSoup object 
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    song={}
    
    song['Title'] = [];
    song['Artist'] = [];
    song['Lyrics'] = [];
    song['Comments']=[];

    # Extract title of the song
    for title in soup.findAll('title'):    
        song['Title'].append(title.text.strip())

    # Extract user comments on the song
    for div in soup.findAll('div',attrs={'class':'rich_text_formatting'}):
        comments = div.text.strip().split("\n")

        for comment in comments: 
            if comment != "":
                song['Comments'].append(comment);

    # Extract the lyrics of the song
    for div in soup.findAll('div', attrs = {'class':'lyrics'}):
        song['Lyrics'].append(div.text.strip().split("\n"));
   
    df_length = len(df)
    df.loc[df_length] = [song['Title'],song['Artist'],song['Lyrics'],song['Comments']]

In [11]:
df.head()

,Title,Artist,Lyrics,Comments
0,[Lana Del Rey – Young and Beautiful Lyrics | G...,[],"[[[Verse 1], I've seen the world, done it all,...",[From the soundtrack for the 2013 film version...
1,[Lana Del Rey – Love Lyrics | Genius Lyrics],[],"[[[Verse 1], Look at you, kids, with your vint...",[“Love” sees Del Rey treading her familiar the...
2,[Lana Del Rey – Summertime Sadness Lyrics | Ge...,[],"[[[Chorus], Kiss me hard before you go, Summer...",[“Summertime Sadness” explores the atmosphere ...
3,[Lana Del Rey – Lust for Life Lyrics | Genius ...,[],"[[[Verse 1: Lana Del Rey], Climb up the H of t...",[Not to be confused with the classic Iggy Pop ...
4,[The Weeknd & Lana Del Rey – Prisoner Lyrics |...,[],"[[[Verse 1: The Weeknd], You bring good to my ...",[“Prisoner” debuted during The Weeknd’s show i...


In [12]:
first_hit

{'highlights': [],
 'index': 'song',
 'type': 'song',
 'result': {'annotation_count': 20,
  'api_path': '/songs/3315890',
  'full_title': "God's Plan by\xa0Drake",
  'header_image_thumbnail_url': 'https://images.genius.com/921b81a80b404f357609b3884052312e.300x169x1.jpg',
  'header_image_url': 'https://images.genius.com/921b81a80b404f357609b3884052312e.1000x563x1.jpg',
  'id': 3315890,
  'lyrics_owner_id': 1507505,
  'lyrics_state': 'complete',
  'path': '/Drake-gods-plan-lyrics',
  'pyongs_count': 852,
  'song_art_image_thumbnail_url': 'https://images.genius.com/efde8de93f83871f717e1019d453245c.300x300x1.png',
  'song_art_image_url': 'https://images.genius.com/efde8de93f83871f717e1019d453245c.1000x1000x1.png',
  'stats': {'unreviewed_annotations': 0,
   'concurrents': 2,
   'hot': False,
   'pageviews': 8400335},
  'title': 'God’s Plan',
  'title_with_featured': "God's Plan",
  'url': 'https://genius.com/Drake-gods-plan-lyrics',
  'primary_artist': {'api_path': '/artists/130',
   'head

In [13]:
df['Artist'] = df['Artist'].replace('[]','')
df['Title'] = df['Title'].replace('\xa0',' ')
df['Title'] = df['Title'].str.split('|')[0]

df['Artist'] = df['Title'].str.split('–')[0]
df['Title'] = df['Title'].str.split('–')[1]

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [ ]:
 for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    